Always Run after 1.30 AM IST.

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from arch import arch_model
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

In [2]:
# Getting the current date
from datetime import date
today = date.today()
today

datetime.date(2022, 5, 20)

In [3]:
raw_data = yfinance.download (tickers = "^NSEI ^FTSE ^N225 ^NYA", start = "2020-03-23", end = today,
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [4]:
df_comp = raw_data.copy()

In [5]:
df_comp['nifty'] = df_comp['^NSEI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]
df_comp['nyse'] = df_comp['^NYA'].Close[:]

In [6]:
df_comp = df_comp.iloc[1:]
del df_comp['^NSEI']
del df_comp['^FTSE']
del df_comp['^N225']
del df_comp['^NYA']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [7]:
df_comp.tail()

,nifty,ftse,nikkei,nyse
,,,,
Date,,,,
2022-05-13,15782.150391,7418.200195,26427.650391,15257.360352
2022-05-16,15842.299805,7464.799805,26547.050781,15259.820312
2022-05-17,16259.299805,7518.399902,26659.750000,15541.910156
2022-05-18,16240.299805,7438.100098,26911.199219,15044.519531
2022-05-19,15809.400391,7302.700195,26402.839844,15035.870117


In [8]:
df_comp['ret_nifty'] = df_comp.nifty.pct_change(1).mul(100)
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1).mul(100)
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1).mul(100)
df_comp['ret_nyse'] = df_comp.nyse.pct_change(1).mul(100)

In [9]:
df_comp['norm_ret_nifty'] = df_comp.nifty.div(df_comp.ret_nifty[1])*100
df_comp['norm_ret_ftse'] = df_comp.ret_ftse.div(df_comp.ret_ftse[1])*100
df_comp['norm_ret_nikkei'] = df_comp.ret_nikkei.div(df_comp.ret_nikkei[1])*100
df_comp['norm_ret_nyse'] = df_comp.ret_nyse.div(df_comp.ret_nyse[1])*100

In [10]:
df_comp.head()

,nifty,ftse,nikkei,nyse,ret_nifty,ret_ftse,ret_nikkei,ret_nyse,norm_ret_nifty,norm_ret_ftse,norm_ret_nikkei,norm_ret_nyse
,,,,,,,,,,,,
Date,,,,,,,,,,,,
2020-03-24,7801.049805,5446.000000,18092.349609,9658.320312,NaN,NaN,NaN,NaN,117756.193991,NaN,NaN,NaN
2020-03-25,8317.849609,5688.200195,19546.630859,9961.379883,6.624747,4.447304,8.038101,3.137808,125557.243796,100.000000,100.000000,100.000000
2020-03-26,8641.450195,5815.700195,18664.599609,10536.280273,3.890436,2.241482,-4.512446,5.771293,130441.967561,50.400920,-56.138219,183.927515
2020-03-27,8660.250000,5510.299805,19389.429688,10187.209961,0.217554,-5.251309,3.883448,-3.313032,130725.749039,-118.078472,48.313010,-105.584265
2020-03-30,8281.099609,5563.700195,19084.970703,10434.740234,-4.378054,0.969101,-1.570232,2.429814,125002.505621,21.790759,-19.534861,77.436669


In [11]:
df_comp.tail()

,nifty,ftse,nikkei,nyse,ret_nifty,ret_ftse,ret_nikkei,ret_nyse,norm_ret_nifty,norm_ret_ftse,norm_ret_nikkei,norm_ret_nyse
,,,,,,,,,,,,
Date,,,,,,,,,,,,
2022-05-13,15782.150391,7418.200195,26427.650391,15257.360352,-0.163522,2.556238,2.636751,2.185109,238230.239458,57.478379,32.803162,69.638054
2022-05-16,15842.299805,7464.799805,26547.050781,15259.820312,0.381123,0.628179,0.451801,0.016123,239138.189829,14.124949,5.620743,0.513833
2022-05-17,16259.299805,7518.399902,26659.750000,15541.910156,2.632194,0.718038,0.424526,1.848579,245432.769934,16.145465,5.281426,58.913067
2022-05-18,16240.299805,7438.100098,26911.199219,15044.519531,-0.116856,-1.068044,0.943179,-3.200318,245145.966524,-24.015532,11.733857,-101.992163
2022-05-19,15809.400391,7302.700195,26402.839844,15035.870117,-2.653273,-1.820356,-1.889025,-0.057492,238641.575927,-40.931672,-23.500892,-1.832238


In [12]:
size = int(len(df_comp)*0.96)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [13]:
# def predict_dates(num_prediction):
#     last_date = df_test['Date'].values[-1]
#     prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
#     return prediction_dates

In [14]:
# num_prediction = 7

In [15]:
# forecast_dates = predict_dates(num_prediction)

In [16]:
col = df.columns

In [17]:
df_test.head()

,nifty,ftse,nikkei,nyse,ret_nifty,ret_ftse,ret_nikkei,ret_nyse,norm_ret_nifty,norm_ret_ftse,norm_ret_nikkei,norm_ret_nyse
,,,,,,,,,,,,
Date,,,,,,,,,,,,
2022-04-19,16958.650391,7601.299805,26985.089844,16658.900391,-1.251918,-0.198258,0.691720,1.170211,255989.408505,-4.457929,8.605513,37.293906
2022-04-20,17136.550781,7629.200195,27217.849609,16761.349609,1.049024,0.367048,0.862550,0.614982,258674.800015,8.253261,10.730763,19.599091
2022-04-21,17392.599609,7628.000000,27553.060547,16499.330078,1.494168,-0.015732,1.231585,-1.563236,262539.835648,-0.353733,15.321841,-49.819376
2022-04-22,17171.949219,7521.700195,27105.259766,16056.870117,-1.268645,-1.393548,-1.625231,-2.681684,259209.136466,-31.334656,-20.219088,-85.463617
2022-04-25,16953.949219,7380.500000,26590.779297,16054.759766,-1.269512,-1.877238,-1.898084,-0.013143,255918.444709,-42.210688,-23.613583,-0.418859


In [18]:
df_test.tail()

,nifty,ftse,nikkei,nyse,ret_nifty,ret_ftse,ret_nikkei,ret_nyse,norm_ret_nifty,norm_ret_ftse,norm_ret_nikkei,norm_ret_nyse
,,,,,,,,,,,,
Date,,,,,,,,,,,,
2022-05-13,15782.150391,7418.200195,26427.650391,15257.360352,-0.163522,2.556238,2.636751,2.185109,238230.239458,57.478379,32.803162,69.638054
2022-05-16,15842.299805,7464.799805,26547.050781,15259.820312,0.381123,0.628179,0.451801,0.016123,239138.189829,14.124949,5.620743,0.513833
2022-05-17,16259.299805,7518.399902,26659.750000,15541.910156,2.632194,0.718038,0.424526,1.848579,245432.769934,16.145465,5.281426,58.913067
2022-05-18,16240.299805,7438.100098,26911.199219,15044.519531,-0.116856,-1.068044,0.943179,-3.200318,245145.966524,-24.015532,11.733857,-101.992163
2022-05-19,15809.400391,7302.700195,26402.839844,15035.870117,-2.653273,-1.820356,-1.889025,-0.057492,238641.575927,-40.931672,-23.500892,-1.832238


### ARMAX

In [19]:
start_date = "2020-03-25"
end_date = today
model_ret_armax = ARIMA(df.ret_nifty[1:], exog = df[["ret_ftse","ret_nikkei","ret_nyse"]][1:], order = (1,0,1))

results_ret_armax = model_ret_armax.fit()

df_pred_armax = results_ret_armax.predict(start = start_date, end = end_date, 
                                          exog = df_test[["ret_ftse","ret_nikkei","ret_nyse"]][start_date:end_date]) 

df_pred_armax[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.ret_nifty[start_date:end_date].plot(color = "blue")
plt.title("Predictions vs Actual (Returns)", size = 24)
plt.show()

ValueError: Provided exogenous values are not of the appropriate shape. Required (24, 3), got (23, 3).

Blue is actual while Red is predictions

In [ ]:
# Lets zoom in a bit

start_date = "2022-04-13"
end_date = today
model_ret_armax = ARIMA(df.ret_nifty[1:], exog = df[["ret_ftse","ret_nikkei","ret_nyse"]][1:], order = (1,0,1))

results_ret_armax = model_ret_armax.fit()

df_pred_armax = results_ret_armax.predict(start = start_date, end = end_date, 
                                          exog = df_test[["ret_ftse","ret_nikkei","ret_nyse"]][start_date:end_date]) 

df_pred_armax[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.ret_nifty[start_date:end_date].plot(color = "blue")
plt.title("Predictions vs Actual (Returns)", size = 24)
plt.show()

### SARIMAX

In [ ]:
start_date = "2020-03-25"
end_date = today
model_ret_sarimax = SARIMAX(df.ret_nifty[1:], exog = df[["ret_ftse","ret_nikkei","ret_nyse"]][1:], 
                            order = (3,0,4), seasonal_order = (3,0,2,5))
results_ret_sarimax = model_ret_sarimax.fit()

df_pred_sarimax = results_ret_sarimax.predict(start = start_date, end = end_date, 
                                              exog = df_test[["ret_ftse","ret_nikkei","ret_nyse"]][start_date:end_date]) 

df_pred_sarimax[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.ret_nifty[start_date:end_date].plot(color = "blue")
plt.title("Predictions vs Actual", size = 24)
plt.show()

In [ ]:
# Lets zoom in a bit

start_date = "2022-04-13"
end_date = today
model_ret_sarimax = SARIMAX(df.ret_nifty[1:], exog = df[["ret_ftse","ret_nikkei","ret_nyse"]][1:], order = (1,0,1))

results_ret_sarimax = model_ret_sarimax.fit()

df_pred_sarimax = results_ret_sarimax.predict(start = start_date, end = end_date, 
                                          exog = df_test[["ret_ftse","ret_nikkei","ret_nyse"]][start_date:end_date]) 

df_pred_sarimax[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.ret_nifty[start_date:end_date].plot(color = "blue")
plt.title("Predictions vs Actual (Returns)", size = 24)
plt.show()